In [ ]:
from adabas.api import *
from adabas.datamap import *
a=Adabas(rbl=256,fbl=64,sbl=32,vbl=128,ibl=0)
a.cb.dbid=12              
a.cb.fnr=11 
dicper={}

In [ ]:
#campo elementar de um grupo periódico
a.cb.isn=201
gsn = 'AQ'        # grupo short name     - FDT
esn = 'AR'        # elementar short name - FDT
lgn = 'curr_code' # elementar long-name  - DDM
lsn = 3          # length elementar     - DDM
upk = ''         # unpack para colunas packeadas
a.fb.value='{}C,002,B.'.format(gsn)
a.get(isn=a.cb.isn)
if a.rb.value:
    occ=int(a.rb.value.encode('hex') ) # count occurs group
    exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
    print '{:08} - occurs {} times'.format(a.cb.isn, occ)
    row.buffer = a.rb
    a.fb.value='{}1-{},{}{}.'.format(esn, occ, lsn, upk) 
    a.get(isn=a.cb.isn)
    print eval("""'dicper["{}"]= [{}]'.format({})""".format( lgn, '{}, ' * occ, ('row.{ln}{}, '* occ).format(ln=lgn,*range(1,occ+1))))

In [ ]:
#campo elementar de um grupo periódico
a.cb.isn=201
gsn = 'AQ'        # grupo short name     - FDT
esn = 'AS'        # elementar short name - FDT
lgn = 'salary'    # elementar long-name  - DDM
lsn = 9           # length elementar     - DDM
upk = ',U'        # unpack para colunas packeadas

a.fb.value='{}C,002,B.'.format(gsn)
a.get(isn=a.cb.isn)
if a.rb.value:
    occ=int(a.rb.value.encode('hex') ) # count occurs group
    exec ("row = Datamap('Rows', {})".format("String('{lg}{}', {l})," * occ).format(lg=lgn, *range(1,occ+1), l=lsn))
    print '{:08} - occurs {} times'.format(a.cb.isn, occ)
    row.buffer = a.rb
    a.fb.value='{}1-{},{}{}.'.format(esn, occ, lsn, upk) 
    a.get(isn=a.cb.isn)
    print eval("""'dicper["{}"]= [{}]'.format({})""".format( lgn, '{}, ' * occ, ('row.{ln}{}, '* occ).format(ln=lgn,*range(1,occ+1))))

In [ ]:
a.cb.isn=200
try:
    for c in xrange(10):
        a.fb.value='AQC,002,B.'
        a.readByIsn(getnext=1)
        if a.rb.value:
            aqc=int(a.rb.value.encode('hex') )
            aql=9
            aqu=',9,U'
            exec ("row = Datamap('Rows', {})".format("String('as{}', {l})," * aqc).format(*range(1,aqc+1), l=aql))
            print '{:08} - occurs {} times'.format(a.cb.isn, aqc)
            row.buffer = a.rb
            a.fb.value='AS1-{}{}.'.format(int(aqc), aqu) 
            a.get(isn=a.cb.isn)
            print eval("'{:08} {}'.format({})".format(a.cb.isn, '- {} ' * aqc, ('row.as{}, '* aqc).format(*range(1,aqc+1))))
            
except DataEnd:
    print 'acabou'

In [ ]:
a.cb.isn=3
a.fb.value='AT1C,2,B.'
a.get(isn=a.cb.isn)
a.rb.value.encode('hex')

In [ ]:
a.cb.isn=1
a.fb.value='AT(1,1),9,U.'
a.get(isn=a.cb.isn)
a.rb.value

In [ ]:
a.cb.isn=201
a.fb.value='AS1-3,9,U.'
a.get(isn=a.cb.isn)
a.rb.value